<h1>Task 2 Data Wrangling</h1>

In [1]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from string import punctuation
import re

pd.set_option('display.max_rows', 10)

<h2>MIMIC III</h2>

First we load the MIMIC III dataset, which will function as our training/development dataset. Before we can use the data for building our weak supervision models we still have to do some preprocessing of certain columns.

In [2]:
df_notes = pd.read_csv('./data/train/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv', low_memory=False)

In [5]:
df_notes.shape

(2083180, 11)

In [5]:
df_notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


<h2>Preprocessing</h2>

In [3]:
notes_text = df_notes.TEXT

In [4]:
notes_text = notes_text.str.replace(r'\[\*\*\d{4}-\d{1,2}-\d{1,2}\*\*\]', '') # extracting date values

/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [5]:
punctuation = list(punctuation)
stopwords = stopwords.words('english')
sentence_ids = {'.', '!', '?', '\''}
punctuation = [p for p in punctuation if p not in sentence_ids] # punctuation without sentence identifiers
remove_words = stopwords + punctuation

In [6]:
notes_text = notes_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (remove_words)]))

For feeding the MIMIC III data to our labeling functions we opted for a n-gram approach - i. e. every note entry is split into x n-grams corresponding to x number of rows. As a starting point we opted for a value of 3 for our n-gram parameter, which can of course still be adapted if a given subtask requires finetuning.

In [ ]:
notes_ngrams = notes_text.str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=3)))).explode()

In [ ]:
notes_ngrams.to_csv('./data/train/notes_preprocessed.csv', index=False)

<h2>Labeling Functions</h2>

As a next step we load a text file containing a list of the most important clinical departments. This list was scraped from different sources, which are also provided in the text file. The input from the text file is then transformed into a dictionary, consisting of various key-value pairs, with each key denoting a different clinical department while the corresponding value refers to a list of different possible names and acronomys for the respective key. This dictionary will later be used as an input for our labeling functions.

In [1]:
with open("./data/keywords/departments_list.txt") as dataFile:
    departments = {}
    for line in dataFile:
        if line.strip() == '': # exclude unnecessary lines
            break
        else:
            line = line.split(':')
            key, value = line[0], line[1:]
            value = [i.replace(';', ',') for i in value]
            value = [i.split(',') for i in value]
            [[value]] = [value]
            value = [i.strip() for i in value]
            departments[key] = value

departments

{'Department of Admissions': ['Admission',
  'Admissions',
  'Admitting Department'],
 'Department of Anaesthesia, Intensive Care Medicine and Pain Medicine': ['Anesthetics',
  'Anesthesiology'],
 'Department of Blood Group Serology and Transfusion Medicine': ['Serology',
  'Transfusion Medicine'],
 'Department of Cardiac Surgery': ['Cardiology'],
 'Department of Clinical Pathology': ['Clinical Pathology',
  'Medical Laboratory'],
 'Department of Dermatology': ['Dermatology'],
 'Department of Ear, Nose and Throat Diseases': ['Otolaryngology',
  'ENT Department',
  'Ear',
  'Nose and Throat Diseases'],
 'Department of Emergency Medicine': ['Accident and emergency',
  'A&E',
  'Casualty Department'],
 'Department of Gastroenterology': ['Gastroenterology'],
 'Department of General Surgery': ['General Surgery', 'Surgery'],
 'Department of Geriatry': ['Geriatric Department', 'Geriatrics'],
 'Department of Haematology': ['Haematology'],
 'Department of Hospital Hygiene and Infection Control'

In [8]:
departments_keywords = list(departments.values())
departments_keywords = [item for i in departments_keywords for item in i]

In [9]:
departments_keywords

['Admission',
 'Admissions',
 'Admitting Department',
 'Anesthetics',
 'Anesthesiology',
 'Serology',
 'Transfusion Medicine',
 'Cardiology',
 'Clinical Pathology',
 'Medical Laboratory',
 'Dermatology',
 'Otolaryngology',
 'ENT Department',
 'Ear',
 'Nose and Throat Diseases',
 'Accident and emergency',
 'A&E',
 'Casualty Department',
 'Gastroenterology',
 'General Surgery',
 'Surgery',
 'Geriatric Department',
 'Geriatrics',
 'Haematology',
 'Central Sterile Services Department',
 'CSSD',
 'Sterile Processing Department',
 'SPD',
 'Sterile Processing',
 'Infection Control',
 'Pharmacy',
 'Medicine Department',
 'Neurology',
 'Nursing Department',
 'Nutrition Department',
 'Dietetics',
 'Gynaecology',
 'Gynecology',
 'Obstetrics',
 'Ophthalmology',
 'Optometry',
 'Oral Surgery',
 'Maxillofacial Surgery',
 'Orthopedics',
 'Orthopaedics',
 'Pediatrics',
 'Paediatrics',
 'Plastic Surgery Department',
 'Aesthetic Surgery Department',
 'Reconstructive Surgery Department',
 'Physiotherapy',